In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
path = "Consumer_Complaints.csv"

In [32]:
# creating a generator object which spits out DataFrames (df) of size 5000 each upon calling the next() function
# because the whole file is 500 MB so, it takes a while to load it completely and it also eats up a lot of memory

# this generator can also be put into a for loop to continuously spit DataFrames

df_reader = pd.read_csv(path,parse_dates=["Date received","Date sent to company"],chunksize=5000)

In [33]:
df = next(df_reader)

In [34]:
# as you can see, the column names are quite messy with '-' (minus) instead of '_' (underscore)
# and also there are '?' (question marks) in the names and white spaces between words
# so, lets first clean the column names by renaming them

df.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [ ]:
# you can do the renaming manually
'''
new_columns = ["date_received","product","sub_product","issue","sub_issue","consumer_complaint_narrative",
              "company_public_response","company","state","zip_code","tags","consumer_consent","submitted_way",
              "date_sent","company_response","timely_response","consumer_disputed","complaint_ID"]

df.columns = new_columns

'''

In [ ]:
# or you can automate it in a simple way
# just take the column name and change it into lowercase, replace white spaces with underscores,
# remove any unusual symbols like '?' and you are done!

new_columns = [name.lower().replace(' ','_').replace('-','_').replace('?','') for name in df.columns]
new_columns

df.columns = new_columns

# but this method has a drawback. that is, the column names are sometimes too long
# eg: 'company_response_to_consumer'
# if you want then you can selectively rename a column like this

df.rename(columns={'company_response_to_consumer' : 'company_response', 
                   'date_sent_to_company' : 'date_sent', 
                   'consumer_complaint_narrative' : 'complaint_narrative'}, 
          inplace=True)

df.columns

In [ ]:
# you can also do the removal of special symbols and selective renaming at the same time
# to do this, you first have to create a function (also called a mapper function) to remove 
# special characters.
# you can also use 'lambda functions' if you want

# instead of '.replace()' you can use regular expressions for more efficiency and flexibility 
# but lets not make this complicated for now

In [ ]:
# using mapper function:

def rename_func(name):
    return name.lower().replace(' ','_').replace('-','_').replace('?','')

# first remove the special characters using pre defined function
df.rename(mapper=rename_func, 
          axis = 1,
          inplace=True)

In [35]:
# or use a lambda function:
df.rename(mapper= lambda x: x.lower().replace(' ','_').replace('-','_').replace('?',''), 
          axis=1, 
          inplace=True)

In [36]:
# then selectively rename
df.rename(columns = {'company_response_to_consumer' : 'company_response', 
                   'date_sent_to_company' : 'date_sent', 
                   'consumer_complaint_narrative' : 'complaint_narrative'},
          inplace=True)

print(df.columns)

# sadly, pandas wont allow you to do both at the same time

Index(['date_received', 'product', 'sub_product', 'issue', 'sub_issue',
       'complaint_narrative', 'company_public_response', 'company', 'state',
       'zip_code', 'tags', 'consumer_consent_provided', 'submitted_via',
       'date_sent', 'company_response', 'timely_response', 'consumer_disputed',
       'complaint_id'],
      dtype='object')


In [37]:
print(df.info())
df.head()

# as you can see, this is one heck of a messy data with null values everywhere
# it may also have many outliers like -99999 and redundant values that have the 
# same meaning like 'Male', 'male', 'm', 'M'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 18 columns):
date_received                5000 non-null datetime64[ns]
product                      5000 non-null object
sub_product                  3495 non-null object
issue                        5000 non-null object
sub_issue                    2029 non-null object
complaint_narrative          1062 non-null object
company_public_response      1356 non-null object
company                      5000 non-null object
state                        4952 non-null object
zip_code                     4952 non-null object
tags                         735 non-null object
consumer_consent_provided    1991 non-null object
submitted_via                5000 non-null object
date_sent                    5000 non-null datetime64[ns]
company_response             5000 non-null object
timely_response              5000 non-null object
consumer_disputed            4922 non-null object
complaint_id                

,date_received,product,sub_product,issue,sub_issue,complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent,company_response,timely_response,consumer_disputed,complaint_id
0,2014-03-12,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,M&T BANK CORPORATION,MI,48382,NaN,NaN,Referral,2014-03-17,Closed with explanation,Yes,No,759217
1,2016-10-01,Credit reporting,NaN,Incorrect information on credit report,Account status,I have outdated information on my credit repor...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,NaN,Consent provided,Web,2016-10-05,Closed with explanation,Yes,No,2141773
2,2016-10-17,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,I purchased a new car on XXXX XXXX. The car de...,NaN,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,2016-10-20,Closed with explanation,Yes,No,2163100
3,2014-06-08,Credit card,NaN,Bankruptcy,NaN,NaN,NaN,AMERICAN EXPRESS COMPANY,ID,83854,Older American,NaN,Web,2014-06-10,Closed with explanation,Yes,Yes,885638
4,2014-09-13,Debt collection,Credit card,Communication tactics,Frequent or repeated calls,NaN,NaN,"CITIBANK, N.A.",VA,23233,NaN,NaN,Web,2014-09-13,Closed with explanation,Yes,Yes,1027760


In [38]:
# later convert it into generator object

# extract the columns that have null values
# decide the relevancy of these columns 
# remove columns that are mostly empty
null_cols = [col for col in df.columns if df[col].isnull().any()]

# find the unique elements of each categorical column and try to minimize 
# redundant values with same meaning
# also change the datatype from 'object' to 'categorical'


# substitute 'product' values where ever 'sub_product' is null
# substitute 'issue' values where ever 'sub_issue' is null
# maybe apply NLP on 'complaint_narrative' if time is avalilable

In [39]:
null_cols

['sub_product',
 'sub_issue',
 'complaint_narrative',
 'company_public_response',
 'state',
 'zip_code',
 'tags',
 'consumer_consent_provided',
 'consumer_disputed']

In [40]:
# percent of nan values in each column

for col in null_cols:
    print(col," : ",round(df[col].isnull().sum()/df[col].size * 100 ,3))

sub_product  :  30.1
sub_issue  :  59.42
complaint_narrative  :  78.76
company_public_response  :  72.88
state  :  0.96
zip_code  :  0.96
tags  :  85.3
consumer_consent_provided  :  60.18
consumer_disputed  :  1.56


## decide the relevance of the columns with null values

In [42]:
# check if 'product' and 'sub_product' are same in any row

df[['product','sub_product']][df['product'] == df['sub_product']]

# no same values
# so we can fill 'product' values where 'sub_product' is null
# that way, the user of this data will know that it was null previously

,product,sub_product


In [31]:
#df['sub_product'].replace(np.nan, df['product'],inplace=True)

In [60]:
# this method is way faster than replace

df['sub_product'] = np.where(df['sub_product'].isnull(), df['product'], df['sub_product'])

In [62]:
df[['sub_product']][df['sub_product'].isnull()].head()

,sub_product


In [65]:
# check if 'issue' and 'sub_issue' are same in any row

df[['issue','sub_issue']][df['issue'] == df['sub_issue']]

# no same values
# so we can fill 'issue' values where 'sub_issue' is null
# that way, the user of this data will know that it was null previously

,issue,sub_issue


In [68]:
df['sub_issue'] = np.where(df['sub_issue'].isnull(), df['issue'], df['sub_issue'])

### are these two columns relevant? (apply nlp?) (too many null values)

In [74]:
# these two columns are irrelevant (i think so)

df[['complaint_narrative', 'company_public_response']].head(10)

,complaint_narrative,company_public_response
0,NaN,NaN
1,I have outdated information on my credit repor...,Company has responded to the consumer and the ...
2,I purchased a new car on XXXX XXXX. The car de...,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,Company believes it acted appropriately as aut...
7,An account on my credit report has a mistaken ...,Company chooses not to provide a public response
8,NaN,Company believes it acted appropriately as aut...
9,NaN,NaN


### how can USA have 58 states?

In [154]:
df['state'].unique().shape

(58,)

In [157]:
df['zip_code'].unique().shape

(3455,)

### there is probably an association between state and zip_code

### tags column is completely irrelevant

In [161]:
df['tags'].unique()

array([nan, 'Older American', 'Servicemember',
       'Older American, Servicemember'], dtype=object)

In [163]:
df.drop(columns=['tags'], axis=1, inplace=True)

### there is a relation between these 3 columns. That is, in each row, either they are all filled up or all null

### check if there is a strong correlation between 'company_public_response' and 'consumer_consent_provided'
look at the semantics of both

In [169]:
df[['complaint_narrative', 'company_public_response', 'consumer_consent_provided']].head(10)

,complaint_narrative,company_public_response,consumer_consent_provided
0,NaN,NaN,NaN
1,I have outdated information on my credit repor...,Company has responded to the consumer and the ...,Consent provided
2,I purchased a new car on XXXX XXXX. The car de...,NaN,Consent provided
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,Company believes it acted appropriately as aut...,Consent not provided
7,An account on my credit report has a mistaken ...,Company chooses not to provide a public response,Consent provided
8,NaN,Company believes it acted appropriately as aut...,NaN
9,NaN,NaN,NaN


array(['No', 'Yes', nan], dtype=object)

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
date_received                5000 non-null datetime64[ns]
product                      5000 non-null category
sub_product                  5000 non-null category
issue                        5000 non-null category
sub_issue                    5000 non-null category
complaint_narrative          1062 non-null object
company_public_response      1356 non-null category
company                      5000 non-null category
state                        4952 non-null category
zip_code                     4952 non-null object
consumer_consent_provided    1991 non-null object
submitted_via                5000 non-null object
date_sent                    5000 non-null datetime64[ns]
company_response             5000 non-null object
timely_response              5000 non-null object
consumer_disputed            4922 non-null object
complaint_id                 5000 non-null int64
dtypes: catego

## relevancy check ends here

## change the dtype of columns

In [120]:
for col in df.columns:
    print(df[col].unique().shape, col)

(1617,) date_received
(15,) product
(56,) sub_product
(102,) issue
(151,) sub_issue
(1061,) complaint_narrative
(11,) company_public_response
(635,) company
(58,) state
(3455,) zip_code
(4,) tags
(5,) consumer_consent_provided
(6,) submitted_via
(1527,) date_sent
(7,) company_response
(2,) timely_response
(3,) consumer_disputed
(5000,) complaint_id


In [ ]:
# the list is incomplete
cat_cols = ['product', 'sub_product', 'issue', 'sub_issue', 'company_public_response', 'company', 'state']

# automate the selection by looking at the percentage of non null values and the unique values 

## dtype changing ends here

## check for outliers

## outliers check ends here

In [ ]:
df.columns

In [ ]:
categorical_cols = ["product","sub_product","issue","sub_issue","company_public_response","company","state","zip_code",
                    "tags","consumer_consent","submitted_way","company_response","timely_response","consumer_disputed"]

In [ ]:
for col in categorical_cols :
    df[col] = df[col].astype("category")

In [ ]:
df.info()

In [ ]:
# # percent of nan values in each column

# for col in null_cols:
#     print(col," : ",round(df[col].isnull().sum()/df[col].size * 100 ,3))

In [ ]:
df.head()

In [ ]:
# the top 10 zip codes with most complaints
df["zip_code"].value_counts()[:10]

In [ ]:
list(df["zip_code"].value_counts()[:10].index)

In [ ]:
# states corresponding the above zip codes

list(df["state"][df["zip_code"].isin(list(df["zip_code"].value_counts()[:10].index))].unique())

# find out if these states have any spatial relation (use folium for maps)